# Mp 1 0 1_Fill Ntuples
Fill n-tuples in distinct workers.
This tutorial illustrates the basics of how it's possible with ROOT to
offload heavy operations on multiple processes and how it's possible to write
simultaneously multiple files. The operation performed in this case is the
creation of random gaussian numbers.




**Author:** Danilo Piparo  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:15 AM.</small></i>

Some useful constants and functions

Total amount of numbers

In [1]:
const UInt_t nNumbers = 20000000U;

The number of workers

In [2]:
const UInt_t nWorkers = 4U;

We split the work in equal parts

In [3]:
const auto workSize = nNumbers / nWorkers;

  A simple function to fill ntuples randomly
 

In [4]:
%%cpp -d
void fillRandom(TNtuple &ntuple, TRandom3 &rndm, UInt_t n)
{
   for (auto i : ROOT::TSeqI(n))
      ntuple.Fill(rndm.Gaus());
}

No nuisance for batch execution

In [5]:
gROOT->SetBatch();

---------------------------------------
 Perform the operation sequentially

Create a random generator and and ntuple to hold the numbers

In [6]:
TRandom3 rndm(1);
TFile ofile("mp101_singleCore.root", "RECREATE");
TNtuple randomNumbers("singleCore", "Random Numbers", "r");
fillRandom(randomNumbers, rndm, nNumbers);
randomNumbers.Write();
ofile.Close();

---------------------------------------
 We now go MP!

We define our work item

In [7]:
auto workItem = [](UInt_t workerID) {
   // One generator, file and ntuple per worker
   TRandom3 workerRndm(workerID); // Change the seed
   TFile ofile(Form("mp101_multiCore_%u.root", workerID), "RECREATE");
   TNtuple workerRandomNumbers("multiCore", "Random Numbers", "r");
   fillRandom(workerRandomNumbers, workerRndm, workSize);
   workerRandomNumbers.Write();
   return 0;
};

Create the pool of workers

In [8]:
ROOT::TProcessExecutor workers(nWorkers);

Fill the pool with work

In [9]:
workers.Map(workItem, ROOT::TSeqI(nWorkers));

return 0;